# Zurich Instruments LabOne Python API Example
# Connect to a Zurich Instruments device

Demonstrate how to connect to a Zurich Instruments HDAWG and upload and run an
AWG program using the command table.

Requirements:
* LabOne Version >= 22.08
* Instruments:
    1 x HDAWG Instrument

---

In [ ]:
import textwrap
import json
import jsonschema
import numpy as np

import zhinst.core
import zhinst.utils

Set up the connection. The connection is always through a session to a
Data Server. The Data Server then connects to the device.

The LabOne Data Server needs to run within the network, either on localhost when
starting LabOne on your local computer or a remote server. (The MFLI/IA devices
have a Data Server running on the device itself, please see the
[user manual](https://docs.zhinst.com/mfli_user_manual/getting_started/device_connectivity_instrument.html)
for detailed explanation.)

In [ ]:
device_id = "dev8123"  # Device serial number available on its rear panel.
interface = "1GbE"  # For Ethernet connection.
# interface = "USB" # For all instruments connected to the host computer via USB.

server_host = "localhost"
server_port = 8004
api_level = 6  # Maximum API level supported for all instruments.

# Create an API session to the Data Server.
daq = zhinst.core.ziDAQServer(server_host, server_port, api_level)
# Establish a connection between Data Server and Device.
daq.connectDevice(device_id, interface)

## Basic configuration
`system/awg/channelgrouping` : Configure how many independent sequencers
should run on the AWG and how the outputs are grouped by sequencer.

0. 4x2 with HDAWG8; 2x2 with HDAWG4.
1. 2x4 with HDAWG8; 1x4 with HDAWG4.
2. 1x8 with HDAWG8.

Configure the HDAWG to use one sequencer with the same waveform on all output channels.

In [ ]:
daq.setInt(f"/{device_id}/system/awg/channelgrouping", 0)

Some basic device configuration to output the generated wave on Wave outputs 1 and 2

In [ ]:
amplitude = 1.0
exp_setting = [
    [f"/{device_id}/sigouts/0/on", 1],
    [f"/{device_id}/sigouts/1/on", 1],
    [f"/{device_id}/sigouts/0/range", 1],
    [f"/{device_id}/sigouts/1/range", 1],
    [f"/{device_id}/awgs/0/outputs/0/amplitude", amplitude],
    [f"/{device_id}/awgs/0/outputs/1/amplitude", amplitude],
    [f"/{device_id}/awgs/0/outputs/0/modulation/mode", 0],
    [f"/{device_id}/awgs/0/time", 0],
    [f"/{device_id}/awgs/*/enable", 0],
    [f"/{device_id}/awgs/0/userregs/0", 0],
]
daq.set(exp_setting)

## AWG sequencer program
Define an AWG program as a string stored in the variable awg_program, equivalent to what would
be entered in the Sequence Editor window in the graphical UI. Different to a self-contained
program, this example refers to a command table by the instruction "executeTableEntry", and to
a placeholder waveform p by the instruction "placeholder". Both the command table and the
waveform data for the waveform p need to be uploaded separately before this sequence program
can be run.

In [ ]:
awg_program = textwrap.dedent(
    """\
    // Define placeholder with 1024 samples:
    wave p = placeholder(1024);

    // Assign placeholder to waveform index 10
    assignWaveIndex(p, p, 10);

    while(true) {
      executeTableEntry(0);
    }
    """
)
device_type = daq.getString(f"/{device_id}/features/devtype")
samplerate = daq.getDouble(f"/{device_id}/system/clocks/sampleclock/freq")

elf, compiler_info = zhinst.core.compile_seqc(
    awg_program, devtype=device_type, samplerate=samplerate
)
print(compiler_info)
assert not compiler_info[
    "messages"
], f"There was an error during compilation: {compiler_info['messages']}"

In [ ]:
daq.setVector(f"/{device_id}/awgs/0/elf/data", elf)
assert (
    daq.getDouble(f"/{device_id}/awgs/0/elf/progress") == 100.0
), "Elf file was not uploaded correctly"

## Command Table definition and upload

The structure of the command table is defined in a schema. The schema can be 
read from the device. This example validates the command table against the 
schema before uploading it. This step is not mandatory since the device will
validate the schema as well. However it is helpfull for debugging.

In [ ]:
schema = json.loads(
    daq.get(f"/{device_id}/awgs/0/commandtable/schema", flat=True)[
        f"/{device_id}/awgs/0/commandtable/schema"
    ][0]["vector"]
)
print(f"The device is using the commandtable schema version {schema['version']}")

In [ ]:
ct = {
    "header": {
        "version": "1.0.0",
    },
    "table": [
        {
            "index": 0,
            "waveform": {"index": 10},
            "amplitude0": {"value": 1.0},
            "amplitude1": {"value": 1.0},
        }
    ],
}

jsonschema.validate(
    instance=ct,
    schema=schema,
    cls=jsonschema.Draft4Validator,
)

In [ ]:
daq.setVector(f"/{device_id}/awgs/0/commandtable/data", json.dumps(ct))
assert (
    daq.getInt(f"/{device_id}/awgs/0/commandtable/status") == 1
), f"The upload of command table failed. \n{ct}"

## Waveform upload

Replace the placeholder waveform with a new one with a Gaussian shape.

The waveform data is uploaded to the index 10 (this is the index assigned with
the assignWaveIndex sequencer instruction)

In [ ]:
x_array = np.linspace(0, 1024, 1024)
x_center = 512
sigma = 150
waveform = np.array(
    np.exp(-np.power(x_array - x_center, 2.0) / (2 * np.power(sigma, 2.0))),
    dtype=float,
)
waveform_native = zhinst.utils.convert_awg_waveform(waveform, -waveform)

In [ ]:
index = 10
daq.setVector(f"/{device_id}/awgs/0/waveform/waves/{index}", waveform_native)

## Enable the AWG 
This is the preferred method of using the AWG: Run in single mode continuous waveform playback
is best achieved by using an infinite loop (e.g., while (true)) in the sequencer program.

In [ ]:
daq.setInt(f"/{device_id}/awgs/0/single", 1)
daq.setInt(f"/{device_id}/awgs/0/enable", 1)